In [ ]:
!pip install ultralytics

Download dataset

In [ ]:
!gdown --id '1PdYzOCwKAqFLx9MAQnzu5vhdpS-EGw98'

In [ ]:
!unzip 'addData.zip'

Download model

In [ ]:
!gdown --id '1-nTvKMqC1rkccqaUFSGOQ1lMqNyDDwFU'

In [8]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('best.pt')
def process_yolov8(source_image_path):
      results = model.predict(source_image_path, save=False, imgsz=640, conf=0.25)
      for result in results:
            boxes = result.boxes

      im0 = cv2.imread(source_image_path)
      max_area = 0
      largest_cropped_image = im0
      h, w = 0, 0
      for bbox in boxes.xyxy:
            area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
            if area > max_area:
                max_area = area
                # Adjust the bounding box coordinates to add a 20px margin
                x1, y1, x2, y2 = bbox
                w = int(x2) - int(x1)
                h = int(y2) - int(y1)
                largest_cropped_image = im0[int(y1):int(y2), int(x1):int(x2)]
      return largest_cropped_image

In [9]:
import cv2
from tqdm import tqdm
import os

def process_video(input_video_path, output_video_path):
    # Khởi tạo đối tượng VideoCapture
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Lấy tổng số frames và FPS của video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / original_fps
    # Tính toán phần trăm và thời điểm bắt đầu, kết thúc để cắt
    if duration > 110:
        start_percent = 20
        end_percent = 80
    elif duration > 60:
        start_percent = 15
        end_percent = 85
    elif duration > 10:
        start_percent = 10
        end_percent = 90
    else:
        start_percent = 0
        end_percent = 100

    start_frame = int((start_percent / 100) * total_frames)
    end_frame = int((end_percent / 100) * total_frames)

    # Lấy kích thước frame của video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Tạo đối tượng VideoWriter với FPS là desired_fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, original_fps, (224, 224))

    # Đọc từng frame và ghi frame vào video mới nếu nằm trong khoảng đã tính và theo tỷ lệ FPS mới
    current_frame = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Chỉ xử lý frame nếu nằm trong khoảng thời gian đã định
        if start_frame <= current_frame <= end_frame:
            cv2.imwrite('yolo_process.jpg', frame)
            frame = process_yolov8('yolo_process.jpg')
            frame = cv2.resize(frame, (224, 224))
            out.write(frame)

        current_frame += 1

    # Giải phóng tài nguyên
    cap.release()
    out.release()

    print("Video đã được xử lý và lưu tại: ", output_video_path)

In [10]:
data_dir = "/content/ai4life"
output_dir = "data_test_aug_yolo-ed"

for className in tqdm(os.listdir(data_dir)):
    class_dir = os.path.join(data_dir, className)
    if os.path.isdir(class_dir) and not className.startswith('.'):  # Bỏ qua các thư mục ẩn
        for video_file in os.listdir(class_dir):
            if not video_file.startswith('.'):  # Bỏ qua các tệp ẩn
                input_video_path = os.path.join(class_dir, video_file)
                video_name = video_file.split('.')[0]
                output_video_path = f"{output_dir}/{className}/{video_name}.mp4"
                os.makedirs(f"{output_dir}/{className}", exist_ok=True)

                # Kiểm tra xem video đã tồn tại trong thư mục đầu ra hay không
                if not os.path.exists(output_video_path):
                    # Đọc độ dài của video
                    cap = cv2.VideoCapture(input_video_path)
                    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    fps = int(cap.get(cv2.CAP_PROP_FPS))
                    duration = frame_count / fps  # Độ dài video tính theo giây

                    # Kiểm tra nếu độ dài video lớn hơn 300 giây, thì bỏ qua
                    if duration > 300:
                        print(f"Video {input_video_path} có độ dài lớn hơn 5 phút và đã bị bỏ qua.")
                        continue  # Bỏ qua video này và chuyển sang video tiếp theo
                    else:
                        print(f"Xử lý video {input_video_path}")
                        process_video(input_video_path, output_video_path)
                else:
                    print(f"Video {output_video_path} đã được xử lý trước đó và tồn tại trong thư mục đầu ra.")

  0%|          | 0/2 [00:00<?, ?it/s]

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Speed: 3.2ms preprocess, 132.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /content/yolo_process.jpg: 640x384 1 workout, 137.0ms
Speed: 3.0ms preprocess, 137.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /content/yolo_process.jpg: 640x384 1 workout, 138.7ms
Speed: 2.8ms preprocess, 138.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /content/yolo_process.jpg: 640x384 1 workout, 142.2ms
Speed: 2.7ms preprocess, 142.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /content/yolo_process.jpg: 640x384 1 workout, 149.6ms
Speed: 2.7ms preprocess, 149.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /content/yolo_process.jpg: 640x384 1 workout, 144.2ms
Speed: 2.7ms preprocess, 144.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /content/yolo_proce

 50%|█████     | 1/2 [13:56<13:56, 836.72s/it]

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.

image 1/1 /content/yolo_process.jpg: 384x640 1 workout, 123.8ms
Speed: 3.3ms preprocess, 123.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/yolo_process.jpg: 384x640 1 workout, 122.6ms
Speed: 3.4ms preprocess, 122.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/yolo_process.jpg: 384x640 1 workout, 128.8ms
Speed: 3.4ms preprocess, 128.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/yolo_process.jpg: 384x640 1 workout, 131.3ms
Speed: 3.5ms preprocess, 131.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/yolo_process.jpg: 384x640 1 workout, 125.5ms
Speed: 4.2ms preprocess, 125.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/yolo_process.jpg: 384x640 1 workout, 126.9ms
Speed: 3.3ms preprocess, 126.9ms inference, 1.5ms postprocess p

 50%|█████     | 1/2 [24:40<24:40, 1480.82s/it]


error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
